<a href="https://cbitek.com/"><img src="https://cbitek.com/wp-content/uploads/2019/08/Genome.jpg" width="200" align="center"></a>

# Genome Sequences 
### Author: NhanTV

We will <b>simulate the process of genome sequencing</b> simply without considering which alleles and chromosomes gens belong to.

# Workflow
Firstly, DNA sample is read by DNA sequencer into <b>(k, d)-mers</b> defined later. Then, we will contruct a <b>de Bruijn graph</b> with edges that are (k,d)-mers. Because of the faulty reading process, biologists will consider <b>the maximal non-branching paths</b> from the de Bruijn graph before constructing a genome sequence by applying <b>a Eulerian Path</b> to the de Bruijn graph .
<a href="https://cbitek.com/"><img src="https://cbitek.com/wp-content/uploads/2019/09/1_3.jpg" width="800" align="center"></a>

# 1. Data
Genome Data of the <b>Ecoli bacterium</b> can be downloaded <a href="https://github.com/nhanta/Genome-Sequences/blob/master/ecoli.txt">here</a>

### Import Dataset

In [1]:
import numpy as np
import pandas as pd

For easy considering of the sequencing process on a personal computer, we choose <b>the first 100 nucleotides</b>.

In [10]:
with open ("D:/Data Science/Coursera/Bioinformatics/Paired reads/ecoli.txt", "r") as file:
    text = file.read().replace('\n', '')
    print('Genome length:', len(text))
    text = text[:100]
    print('The first 100 nucleotides:', text)

Genome length: 4639675
The first 100 nucleotides: AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT


# 2. DNA Sequencing Reads (Paired Reads)

Firstly, we will simulate the reading of the genome sequencer. Defining a <b>(k, d)-mers</b> as a paired reads with the <b>first k nucleotides</b>, <b>the last k nucleotides</b> and <b>the gap including d nucleotides unknown</b> between them.
<a href="https://cbitek.com/"><img src="https://cbitek.com/wp-content/uploads/2019/09/paired-reads.jpg" width="400" align="center"></a>

In [12]:
# Deviding genome to k-mers overlaping (k-1) nucleotides 
def get_composition(text, k): 
    pattern = []
    for i in range(len(text)-k +1):
        pattern.append(text[i:i+k])
    return(pattern)

The text is composited into (k, d)-mers so that it's surfix: (k-1, d)-mers equals to the prefix (k-1, d)-mers of the next (k, d)-mers.

In [13]:
def get_composition_d(text, k, d):
    pattern = []
    for i in range(len(text)-2*k-d+1):
        pattern.append(text[i:i+k]+'|'+ text[i+k+d:i+2*k+d])
    return(pattern)

In [187]:
pre_paired_reads = get_composition_d(text, 4, 2)
pre_paired_reads[:5]

['AGCT|TCAT', 'GCTT|CATT', 'CTTT|ATTC', 'TTTT|TTCT', 'TTTC|TCTG']

Since we don't know the order of the (k, d)-mers, so we will rearrange its order by alphabe.

In [188]:
paired_reads= sorted(pre_paired_reads)
paired_reads[:5]

['AAAA|AGAG', 'AAAA|AGTG', 'AAAA|GAGT', 'AAAA|GTGT', 'AAAG|TGTC']

# 3. Contructing the De Bruijn Graph
For genome sequences in the first period, people used Hamilton graph with (k, d) -mers as the vertices of the graph, but this method was problematic. The scientists then used the de Bruijn graph with <b>edges being (k, d)-mers</b>, <b>vertices being (k-1, d)-mers</b>. Genome sequencing became finding <b>the Eulerian paths</b> for the De Bruijn graph. Each (k, d)-mer will be the edge of the de Bruijn graph whose vertices are two (k-1, d)-mers separated from (k, d)-mer initially.

<a href="https://cbitek.com/"><img src="https://cbitek.com/wp-content/uploads/2019/09/DE-BRUIJN-2.jpg" width="850" align="center"></a>

In [189]:
# Getting (k-1, d)-mers from (k, d)-mers.
def paired_vertices(text, k): 
    t_pair = []
    for item in text:
        prefix = get_composition(item[:k], k-1)[0] + '|' + get_composition(item[k+1:], k-1)[0]
        surfix = get_composition(item[:k], k-1)[1] + '|' + get_composition(item[k+1:], k-1)[1]
        t_pair.append([prefix, surfix])
    return(t_pair)

In [190]:
paired_ver = paired_vertices(paired_reads, 4)
paired_ver[:5]

[['AAA|AGA', 'AAA|GAG'],
 ['AAA|AGT', 'AAA|GTG'],
 ['AAA|GAG', 'AAA|AGT'],
 ['AAA|GTG', 'AAA|TGT'],
 ['AAA|TGT', 'AAG|GTC']]

In [191]:
# Getting the De Bruijn from paired_vertices.
def get_DeBruijn_k_d_mer(t): 
    count = []
    for i in range(len(t)): 
        h = [t[i][1]]
        for j in range(len(t)):
            
            # Finding a vertex that can be conected to t[i][0]
            if j > i and t[i][0] == t[j][0]:
                h.append(t[j][1])
                count.append(j)
                
        t[i][1] = h
        
    count = np.unique(count).tolist()
    
    # Glueding the same vertices
    for c in count[::-1]: 
        t.remove(t[c])
    return(t)

The de Bruijn graph is constructed from paired reads.

In [192]:
graph = get_DeBruijn_k_d_mer(paired_ver)
graph[30:35]

[['CGT|TAA', ['GTG|AAA']],
 ['CTC|GTG', ['TCT|TGG']],
 ['CTG|GCA', ['TGA|CAA', 'TGA|CAG']],
 ['CTG|TGG', ['TGA|GGT']],
 ['CTG|CGG', ['TGC|GGG']]]

# 4. Eulerian Path
Eulerian Path was first discussed by Leonhard Euler while solving the famous Seven Bridges of Königsberg problem in 1736. <b>Eulerian path is the path that goes through all the edges of the graph only once</b>. 

<a href="https://cbitek.com/"><img src="https://cbitek.com/wp-content/uploads/2019/09/eulerian-2.jpg" width="850" align="center"></a>

### Finding  Vertices of edges from de Bruijn Graph

In [193]:
# Finding vetexes of edges from the De Bruijn graph
def ver(graph): 
    ver = []
    for edge in graph:
        ver.append(edge[0])
        ver.append(edge[1])
    ver = np.unique(ver)
    return (ver)

### Constructing Eulerian Path

In [220]:
def Construct_Eulerian_Path(graph):
        graph = [(src,dst) for src,dst in graph]
        currentVertex = verifyAndGetStart(graph)
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            # follows a path until it ends
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                # Look for side-trips along the path
                for edge in graph:
                    try:
                        # insert our side-trip after the
                        # "u" vertex that is starts from
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print ("There is no path!")
                    return False
        return path
    
# More new methods for the Graph Class
def degrees(graph):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in graph:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return (inDegree, outDegree)
            
def verifyAndGetStart(graph):
        inDegree, outDegree = degrees(graph)
        start = 0
        end = 0
        vertex = ver(graph)
        i = []
        o = []
        for vert in vertex:
            i.append(inDegree.get(vert,0))
            o.append(outDegree.get(vert,0))
            
        if (np.array(i) - np.array(o)).any() != np.zeros((len(i)), dtype=int).any():
            for vert in vertex:
                ins = inDegree.get(vert,0)
                outs = outDegree.get(vert,0)
                if (ins == outs):
                    continue
                elif (ins - outs == 1):
                    end = vert
                elif (outs - ins == 1):
                    start = vert
                else:
                    start, end = 'no', 'no'
                    break
            if (start != 'no') and (end != 'no'):
                return (start)
            else:
                return('no')
        else:
            return (vertex[0])
        

# 5. Maximal Non-Branching Paths in the De Bruijn Graph

After read breaking, some assemblies still have gaps in (k, d)-mer coverage. This causes the De Bruijn graph to lack some edges, and sequenced gene is incorect. Therefore, biologists often divide the De Bruijn graph into <b>Maximal Non-Branching Paths</b> to check before sequencing entire chromosomes.
The maximal non-branching path includes vertices having an in-degree = 1 and an out-degree = 1, excluding the vertices at the beginning and the end of the path.

<a href="https://cbitek.com/"><img src="https://cbitek.com/wp-content/uploads/2019/09/non_branching_paths_2.jpg" width="920" align="center"></a>


In [195]:
def change_graph(graph):
    ch_graph = []
    for x in graph:
        l = len(x[1])
        for i in range(l):
            if x[1][i] != ',':
                ch_graph.append((x[0], x[1][i]))
    return(ch_graph)

In [196]:
debruijn_graph = change_graph(graph)
debruijn_graph[:5]

[('AAA|AGA', 'AAA|GAG'),
 ('AAA|AGT', 'AAA|GTG'),
 ('AAA|GAG', 'AAA|AGT'),
 ('AAA|GTG', 'AAA|TGT'),
 ('AAA|TGT', 'AAG|GTC')]

In [225]:
#Finding edges of the graph
def get_edge_from_v (v, Graph): 
    ed = []
    for edge in Graph:
        if v == edge[0]:
            ed.append((v, edge[1]))
    return (ed)

#Finding Maximal Non-Branching Paths
def MaximalNonBranchingPaths(Graph): 
    inDegree, outDegree = degrees(Graph)
    Paths = []
    vertex = ver(Graph)
    new_graph = Graph[:]
    
    # Finding in-degree and out-degree of vertex v
    for v in vertex:
        ins = inDegree.get(v,0) 
        outs = outDegree.get(v,0)
        
        # Finding maximal non-braching paths that are not sub-cycles
        # To detect vertices that have in-degree or out-degree difference from 1. 
        if ins != 1 or outs != 1: 
            if outs > 0:
                for edge in get_edge_from_v (v, new_graph): 
                    NonBrachingPath = [edge]
                    w = edge[1]
                    new_graph.remove(edge)
                    
                    # Repeating to get next vertices that have in-degree and out-degree equal to 1.
                    while inDegree.get(w, 0) == 1 and outDegree.get(w, 0) == 1:
                        for w_edge in get_edge_from_v(w, new_graph):
                            NonBrachingPath.append(w_edge)
                            w = w_edge[1]
                            new_graph.remove(w_edge)
                    Paths.append(NonBrachingPath) 
                    
        # Finding sub-cycles in the graph            
        elif ins == 1 and outs == 1: 
            first_remain = get_edge_from_v(v, new_graph)
            
            for edge in first_remain:
                NonBranchingPath = [edge]
                w = edge[1]
                # Finding in-degree and out-degree of vertex w
                ins_2 = inDegree.get(w, 0)
                outs_2 = outDegree.get(w, 0)
                
                # Repeating to get next vertices that have in-degree and out-degree equal to 1
                while  ins_2 == 1 and outs_2 == 1:
                    
                    remain_edge = get_edge_from_v(w, new_graph)
                    
                    # Finding next edges
                    for w_edge in remain_edge:
                        if w_edge != edge:
                            NonBranchingPath.append(w_edge)
                            w = w_edge[1]
                            
                    # Breaking while loop if we recieve a cycle   
                    if NonBranchingPath[0][0] == NonBranchingPath[-1][-1]:
                        Paths.append(NonBranchingPath)
                        for i in NonBranchingPath:
                            new_graph.remove(i)  
                        break
                    # Breaking while loop if we don't recieve a cycle      
                    elif NonBranchingPath[0][0] != NonBranchingPath[-1][-1] and remain_edge == []:    
                        break
            
    showing_path = []
    for p in Paths:
        g = Construct_Eulerian_Path(p)
        path = '->'.join(g)
        showing_path.append(path)
    
    return(showing_path)

In [226]:
Max_NonBranching_Paths = MaximalNonBranchingPaths(debruijn_graph)
Max_NonBranching_Paths

['AGC|TCA->GCT|CAT->CTT|ATT->TTT|TTC->TTT|TCT->TTC|CTG',
 'AAA|AGA->AAA|GAG->AAA|AGT->AAA|GTG->AAA|TGT->AAG|GTC->AGA|TCT->GAG|CTG->AGT|TGA->GTG|GAT->TGT|ATA->GTC|TAG->TCT|AGC->CTG|GCA->TGA|CAA->GAC|AAC->ACT|ACG->CTG|CGG->TGC|GGG->GCA|GGC->CAA|GCA->AAC|CAA->ACG|AAT->CGG|ATA->GGG|TAT->GGC|ATG->GCA|TGT->CAA|GTC->AAT|TCT->ATA|CTC->TAT|TCT->ATG|CTG->TGT|TGT->GTC|GTG->TCT|TGT->CTC|GTG->TCT|TGG->CTG|GGA->TGT|GAT->GTG|ATT->TGT|TTA->GTG|TAA->TGG|AAA->GGA|AAA->GAT|AAA->ATT|AAA->TTA|AAA->TAA|AAG->AAA|AGA',
 'CTG|GCA->TGA|CAG->GAT|AGC->ATA|GCT->TAG|CTT->AGC|TTC->GCA|TCT->CAG|CTG->AGC|TGA->GCT|GAA->CTT|AAC->TTC|ACT->TCT|CTG->CTG|TGG->TGA|GGT->GAA|GTT->AAC|TTA->ACT|TAC->CTG|ACC->TGG|CCT->GGT|CTG->GTT|TGC->TTA|GCC->TAC|CCG->ACC|CGT->CCT|GTG->CTG|TGA->TGC|GAG->GCC|AGT->CCG|GTA->CGT|TAA->GTG|AAA->TGA|AAT',
 'ATT|ACT->TTC|CTG->TCA|TGA->CAT|GAC->ATT|ACT',
 'TTC|CTG->TCT|TGC->CTG|GCA']

In [170]:
print('Total number of contigs is:',len(Max_NonBranching_Paths))

Total number of contigs is: 5


# 6. Genome Sequencing
After checking all maximal non-branching paths of the de Bruijn, biologists will <b>find and correct missing or incorrect errors in nucleotides</b> due to an error-reading sequencer. Suppose we finally have all the paired read perfectly, we will <b>build the genome using the eulerian path in the de Bruijn graph</b>.

In [230]:
Eulerian_Path = Construct_Eulerian_Path(debruijn_graph)
print('Eulerian Path: ', "->".join(Eulerian_Path))

Eulerian Path:  AGC|TCA->GCT|CAT->CTT|ATT->TTT|TTC->TTT|TCT->TTC|CTG->TCA|TGA->CAT|GAC->ATT|ACT->TTC|CTG->TCT|TGC->CTG|GCA->TGA|CAA->GAC|AAC->ACT|ACG->CTG|CGG->TGC|GGG->GCA|GGC->CAA|GCA->AAC|CAA->ACG|AAT->CGG|ATA->GGG|TAT->GGC|ATG->GCA|TGT->CAA|GTC->AAT|TCT->ATA|CTC->TAT|TCT->ATG|CTG->TGT|TGT->GTC|GTG->TCT|TGT->CTC|GTG->TCT|TGG->CTG|GGA->TGT|GAT->GTG|ATT->TGT|TTA->GTG|TAA->TGG|AAA->GGA|AAA->GAT|AAA->ATT|AAA->TTA|AAA->TAA|AAG->AAA|AGA->AAA|GAG->AAA|AGT->AAA|GTG->AAA|TGT->AAG|GTC->AGA|TCT->GAG|CTG->AGT|TGA->GTG|GAT->TGT|ATA->GTC|TAG->TCT|AGC->CTG|GCA->TGA|CAG->GAT|AGC->ATA|GCT->TAG|CTT->AGC|TTC->GCA|TCT->CAG|CTG->AGC|TGA->GCT|GAA->CTT|AAC->TTC|ACT->TCT|CTG->CTG|TGG->TGA|GGT->GAA|GTT->AAC|TTA->ACT|TAC->CTG|ACC->TGG|CCT->GGT|CTG->GTT|TGC->TTA|GCC->TAC|CCG->ACC|CGT->CCT|GTG->CTG|TGA->TGC|GAG->GCC|AGT->CCG|GTA->CGT|TAA->GTG|AAA->TGA|AAT


After finding the Eulerian paths through the (k-1, d)-mers, we will merge them into a genome sequences. For example:

TGTCTC<b>TGTGTGGATTAAAAAAAGAGTGTCTGAGCAACGGGCAATATGT</b> + <b>TGTGTGGATTAAAAAAAGAGTGTCTGAGCAACGGGCAATATGT</b>CTCTGT
--------->TGTCTC<b>TGTGTGGATTAAAAAAAGAGTGTCTGAGCAACGGGCAATATGT</b>CTCTGT

In [214]:
def get_genome_string(text, k):
    st = str()
    it = len(text)
    for i in range(it):
        st = st + text[i][0]
    if k > 1:
        st = st + text[it-1][-k+1:-1] + text[it-1][-1]
    else:
        st = st
    return(st)

In [215]:
def get_pair_string(pair, k):
    pre = []
    sur = []
    
    # if paths is not a cycle
    if pair[-1] != pair[0]:
        for item in pair:
            prefix_pair = item[:k-1]
            surfix_pair = item[k:]
            pre.append(prefix_pair)
            sur.append(surfix_pair)
            
    # if paths is a cycle
    else:
        for item in pair:
            prefix_pair = item[:k-1]
            surfix_pair = item[k:]
            pre.append(prefix_pair)
            sur.append(surfix_pair)
        pre.remove(pre[-1])
        sur.remove(sur[-1])
            
    return(pre, sur)

In [235]:
def get_genome_pair(pair, k, d):
    a = get_genome_string(get_pair_string(pair, k)[0], k-1)
    b = get_genome_string(get_pair_string(pair, k)[1], k-1)
    print('Prefix:', a)
    print('Surfix:', b)
    
    if a[k+d:] == b[:len(a)-k-d]: 
        tx = a[:k+d] + b
    else:
        tx = 'Choosing d value again'
    return(tx)

In [236]:
print('Genome sequences:',get_genome_pair(Eulerian_Path, 4, 2))

Prefix: AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGA
Surfix: TCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT
Genome sequences: AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT


# 7. Conclusion

Switching from reads to paired reads will increase the length of reads, so the de bruijn graph becomes less hassle, but must be consider choosing the values of k and d accordingly so we can get a final genetic sequence.
Genome sequencing is not easy due to the reading errors of the sequencing machine. Separating the maxial non-branching paths from the de bruijn graph can make it easier for biologists to identify where the errors are, from which they will correct them before sequencing whole chromosome.